## Check out PCA

In this notebook, we analyze to see whether using Principal Components Analysis (PCA) would yield us any benefits.

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../common_routines/')
import numpy as np
from relevant_functions import\
    evaluate_model_score_given_predictions,\
    evaluate_model_score,\
    evaluate_neg_model_score,\
    cross_val_score_given_model,\
    fit_pipeline_and_cross_validate, \
    fit_pipeline_and_evaluate_on_validation_set, \
    print_model_stats_from_pipeline, \
    get_validated_transformed_data,\
    evaluate_model_score_given_predictions
from sklearn import linear_model 

In [2]:
# Dump alll the dataframes with one hot encoding.
train_data_one_hot = pd.read_csv('../../cleaned_input/train_data_one_hot.csv')
validation_data_one_hot = pd.read_csv('../../cleaned_input/validation_data_one_hot.to_csv')
test_data_one_hot =pd.read_csv('../../cleaned_input/test_data_one_hot.csv')

# Dump the data frames prior to taking the one hot encoding transformation.
# Remember that we had handled the null values at the stage and hence the model
# does not need to worry about the same.
train_data = pd.read_csv('../../cleaned_input/train_data.csv')
validation_data = pd.read_csv('../../cleaned_input/validation_data.csv')
test_data = pd.read_csv('../../input/test.csv')

### Test the presence of muliticollinearity.

Before, we apply PCA, we need to test whetherthe situation is ripe for the same. We do the same , by checking multicollinearity exists in the current scenario.

#### Let us set up data first.

In [3]:
def get_nothing(column, input_df=None, validation_df=None):
    return []

def get_column_itself(column, input_df=None, validation_df=None):
    return [column]

def get_log_transform(column, input_df=None, validation_df=None):
    return ['Log' + column]
    
def get_one_hot_encoded_transformation(column, input_df=None, validation_df=None):
    
    rel_cols = [col for col in input_df.columns if column + '_' in col and column + '_Val' not in col]
    return rel_cols

def get_null_value_handling_with_log_transform(column, input_df=None, validation_df=None):
    rel_cols = ['Log' + column + '_times_not_missing', column + '_not_missing']
    return rel_cols

def get_null_value_handling_with_no_transform(column, input_df=None, validation_df=None):
    rel_cols = [column + '_times_not_missing', column + '_not_missing']
    return rel_cols

In [4]:
rel_cols= ['Id',
           'MSSubClass',
           'MSZoning',
           'LotFrontage',
           'LotArea',
           'Street',
           'LotShape',
           'LandContour',               
           'Utilities',                              
           'LotConfig',                                             
           'LandSlope',                                                            
           'Neighborhood',        
           'Condition1', 
           'Condition2', 
           'BldgType',
           'HouseStyle',
           'OverallQual',
           'OverallCond',
           'YearBuilt', 
           'YearRemodAdd',
           'RoofStyle', 
           'RoofMatl', 
           'Exterior1st',  
           'Exterior2nd', 
           'MasVnrType', 
           'MasVnrArea',
           'ExterQual',
           'ExterCond',
           'Foundation',
           'BsmtQual',
           'BsmtCond',
           'BsmtExposure',
           'BsmtFinType1',
           'BsmtFinSF1',
           'BsmtFinType2',
           'BsmtUnfSF', 
           'TotalBsmtSF',
           'Heating',
           'HeatingQC',
           'CentralAir',
           'Electrical',
           '1stFlrSF',
           '2ndFlrSF',
           'LowQualFinSF',
           'GrLivArea',
           'BsmtFullBath', 
           'BsmtHalfBath', 
           'FullBath',
           'HalfBath', 
           'BedroomAbvGr', 
           'KitchenAbvGr',
           'KitchenQual',
           'TotRmsAbvGrd',
           'Functional',
           'Fireplaces',
           'FireplaceQu',
           'GarageType', 
           'GarageYrBlt',
           'GarageFinish', 
           'GarageCars', 
           'GarageArea', 
           'GarageQual',
           'GarageCond', 
           'PavedDrive', 
           'WoodDeckSF', 
           'OpenPorchSF',
           'EnclosedPorch',
           '3SsnPorch', 
           'ScreenPorch', 
           'PoolArea', 
           'PoolQC',
           'Fence', 
           'MiscFeature', 
           'MiscVal', 
           'MoSold', 
           'YrSold', 
           'SaleType',
           'SaleCondition',   
           'Alley']

In [5]:
relevant_transform_map={'Id' : get_nothing,
                        'MSSubClass' : get_one_hot_encoded_transformation,
                        'MSZoning' : get_one_hot_encoded_transformation,
                        'LotFrontage': get_nothing,
                        'LotArea': get_log_transform,
                        'Street' : get_one_hot_encoded_transformation,                            
                        'LotShape' : get_one_hot_encoded_transformation,                                                     
                        'LandContour' : get_one_hot_encoded_transformation,                                                     
                        'Utilities' : get_one_hot_encoded_transformation,                                                                                 
                        'LotConfig' : get_one_hot_encoded_transformation,                                                                                                             
                        'LandSlope' : get_one_hot_encoded_transformation,                                                                                                                                         
                        'Neighborhood' : get_one_hot_encoded_transformation,                                                                                                                                                                     
                        'Condition1' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'Condition2' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'BldgType' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'HouseStyle' : get_one_hot_encoded_transformation,  
                        'OverallQual' : get_column_itself,  
                        'OverallCond' : get_column_itself,                              
                        'YearBuilt' : get_column_itself,  
                        'YearRemodAdd' : get_column_itself,                              
                        'RoofStyle' : get_one_hot_encoded_transformation,                                                                                                                                                                     
                        'RoofMatl' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'Exterior1st' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'Exterior2nd' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                        'MasVnrType' : get_one_hot_encoded_transformation,  
                        'MasVnrArea' : get_null_value_handling_with_log_transform,
                        'ExterQual' : get_one_hot_encoded_transformation,
                        'ExterCond' : get_one_hot_encoded_transformation,
                        'Foundation' : get_one_hot_encoded_transformation,
                        'BsmtQual' : get_one_hot_encoded_transformation,
                        'BsmtCond' : get_one_hot_encoded_transformation,
                        'BsmtExposure' : get_one_hot_encoded_transformation,
                        'BsmtFinType1' : get_one_hot_encoded_transformation,   
                        'BsmtFinSF1' : get_log_transform,
                        'BsmtFinType2' : get_one_hot_encoded_transformation,
                        'BsmtUnfSF' : get_column_itself, 
                        'TotalBsmtSF' : get_column_itself,
                        'Heating' : get_one_hot_encoded_transformation,
                        'HeatingQC' : get_one_hot_encoded_transformation,
                        'CentralAir' : get_one_hot_encoded_transformation,
                        'Electrical' : get_one_hot_encoded_transformation,  
                        '1stFlrSF' : get_log_transform,
                        '2ndFlrSF' : get_log_transform,
                        'LowQualFinSF' : get_log_transform,
                        'GrLivArea' : get_log_transform,
                        'BsmtFullBath' : get_column_itself, 
                        'BsmtHalfBath':get_column_itself, 
                        'FullBath' : get_column_itself,
                        'HalfBath' : get_column_itself, 
                        'BedroomAbvGr' : get_column_itself, 
                        'KitchenAbvGr' :get_column_itself,
                        'KitchenQual' : get_one_hot_encoded_transformation,
                        'TotRmsAbvGrd' : get_column_itself,
                        'Functional' : get_one_hot_encoded_transformation, 
                        'Fireplaces' : get_column_itself,  
                        'FireplaceQu' : get_one_hot_encoded_transformation,
                        'GarageType' : get_one_hot_encoded_transformation,
                        'GarageYrBlt' : get_null_value_handling_with_no_transform,
                        'GarageFinish' : get_one_hot_encoded_transformation,
                        'GarageCars': get_column_itself,
                        'GarageArea' : get_log_transform,
                        'GarageQual' : get_one_hot_encoded_transformation,
                        'GarageCond' : get_one_hot_encoded_transformation,
                        'PavedDrive' : get_one_hot_encoded_transformation,
                        'WoodDeckSF' : get_log_transform,
                        'OpenPorchSF' : get_log_transform,
                        'EnclosedPorch' : get_log_transform,
                        '3SsnPorch' : get_log_transform,
                        'ScreenPorch' : get_log_transform, 
                        'PoolArea' : get_log_transform, 
                        'PoolQC' : get_one_hot_encoded_transformation,
                        'Fence' : get_one_hot_encoded_transformation,
                        'MiscFeature' : get_one_hot_encoded_transformation,
                        'MiscVal' : get_log_transform, 
                        'MoSold' : get_one_hot_encoded_transformation, 
                        'YrSold' : get_one_hot_encoded_transformation, 
                        'SaleType' : get_one_hot_encoded_transformation,
                        'SaleCondition' : get_one_hot_encoded_transformation,
                        'Alley' : get_nothing}


In [6]:
def get_rel_X_cols(X_columns, X_column_transform_map, input_df=None, validation_df=None):
    rel_X_cols = list()
    for col in X_columns:
        if col in X_column_transform_map.keys():
            rel_cols = X_column_transform_map.get(col)(col , input_df, validation_df)
        else:
            rel_cols = [col]
        for elem in rel_cols:
            rel_X_cols.append(elem)
    return rel_X_cols

In [7]:
train_validation_data_one_hot = pd.concat([train_data_one_hot, validation_data_one_hot])

In [8]:
train_validation_data = pd.concat([train_data, validation_data])

In [9]:
rel_X_cols = get_rel_X_cols(
    X_columns=rel_cols,
    X_column_transform_map=relevant_transform_map,
    input_df=train_data_one_hot,
    validation_df=validation_data_one_hot)


#### Presence of linearly dependent predictors.
When we do one hot encoding , we would have a set of linearly dependent subset of predictors within our model, causing  the variance inflation factors of the relevant column to shoot up to infinity.



In [10]:
given_col = rel_X_cols[0]
print(given_col)
other_cols = [col for col in rel_X_cols if col != given_col]
print(other_cols)
X = train_validation_data_one_hot[other_cols]
Y = train_validation_data_one_hot[[given_col]]

MSSubClass_120
['MSSubClass_150', 'MSSubClass_160', 'MSSubClass_180', 'MSSubClass_190', 'MSSubClass_20', 'MSSubClass_30', 'MSSubClass_40', 'MSSubClass_45', 'MSSubClass_50', 'MSSubClass_60', 'MSSubClass_70', 'MSSubClass_75', 'MSSubClass_80', 'MSSubClass_85', 'MSSubClass_90', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'LogLotArea', 'Street_Grvl', 'Street_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'Utilities_NoSeWa', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood

In [11]:
my_model = linear_model.LinearRegression()

In [12]:
my_model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
my_model.score(X,Y)

1.0

#### Omit one column to remove linear dependency.

We make sure we omit one column, while returning one hot encoded columns of a particular variable(to prevent linear dependence amongst columns)

In [14]:
def get_one_hot_encoded_transformation(column, input_df=None, validation_df=None):
    
    rel_cols = [col for col in input_df.columns if column + '_' in col and column + '_Val' not in col]
    # Omit one column while returning the column list to avoid linear dependence amonst columns.
    return rel_cols[:-1]


In [15]:
def get_vif_model(given_col = 'MSSubClass_120', 
                  rel_cols=rel_cols,
                  train_data_one_hot = train_data_one_hot,
                  validation_data_one_hot = validation_data_one_hot,
                  train_validation_data_one_hot = train_validation_data_one_hot):
    relevant_transform_map={'Id' : get_nothing,
                            'MSSubClass' : get_one_hot_encoded_transformation,
                            'MSZoning' : get_one_hot_encoded_transformation,
                            'LotFrontage': get_nothing,
                            'LotArea': get_log_transform,
                            'Street' : get_one_hot_encoded_transformation,                            
                            'LotShape' : get_one_hot_encoded_transformation,                                                     
                            'LandContour' : get_one_hot_encoded_transformation,                                                     
                            'Utilities' : get_one_hot_encoded_transformation,                                                                                 
                            'LotConfig' : get_one_hot_encoded_transformation,                                                                                                             
                            'LandSlope' : get_one_hot_encoded_transformation,                                                                                                                                         
                            'Neighborhood' : get_one_hot_encoded_transformation,                                                                                                                                                                     
                            'Condition1' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'Condition2' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'BldgType' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'HouseStyle' : get_one_hot_encoded_transformation,  
                            'OverallQual' : get_column_itself,  
                            'OverallCond' : get_column_itself,                              
                            'YearBuilt' : get_column_itself,  
                            'YearRemodAdd' : get_column_itself,                              
                            'RoofStyle' : get_one_hot_encoded_transformation,                                                                                                                                                                     
                            'RoofMatl' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'Exterior1st' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'Exterior2nd' : get_one_hot_encoded_transformation,                                                                                                                                                                                                 
                            'MasVnrType' : get_one_hot_encoded_transformation,  
                            'MasVnrArea' : get_null_value_handling_with_log_transform,
                            'ExterQual' : get_one_hot_encoded_transformation,
                            'ExterCond' : get_one_hot_encoded_transformation,
                            'Foundation' : get_one_hot_encoded_transformation,
                            'BsmtQual' : get_one_hot_encoded_transformation,
                            'BsmtCond' : get_one_hot_encoded_transformation,
                            'BsmtExposure' : get_one_hot_encoded_transformation,
                            'BsmtFinType1' : get_one_hot_encoded_transformation,   
                            'BsmtFinSF1' : get_log_transform,
                            'BsmtFinType2' : get_one_hot_encoded_transformation,
                            'BsmtUnfSF' : get_column_itself, 
                            'TotalBsmtSF' : get_column_itself,
                            'Heating' : get_one_hot_encoded_transformation,
                            'HeatingQC' : get_one_hot_encoded_transformation,
                            'CentralAir' : get_one_hot_encoded_transformation,
                            'Electrical' : get_one_hot_encoded_transformation,  
                            '1stFlrSF' : get_log_transform,
                            '2ndFlrSF' : get_log_transform,
                            'LowQualFinSF' : get_log_transform,
                            'GrLivArea' : get_log_transform,
                            'BsmtFullBath' : get_column_itself, 
                            'BsmtHalfBath':get_column_itself, 
                            'FullBath' : get_column_itself,
                            'HalfBath' : get_column_itself, 
                            'BedroomAbvGr' : get_column_itself, 
                            'KitchenAbvGr' :get_column_itself,
                            'KitchenQual' : get_one_hot_encoded_transformation,
                            'TotRmsAbvGrd' : get_column_itself,
                            'Functional' : get_one_hot_encoded_transformation, 
                            'Fireplaces' : get_column_itself,  
                            'FireplaceQu' : get_one_hot_encoded_transformation,
                            'GarageType' : get_one_hot_encoded_transformation,
                            'GarageYrBlt' : get_null_value_handling_with_no_transform,
                            'GarageFinish' : get_one_hot_encoded_transformation,
                            'GarageCars': get_column_itself,
                            'GarageArea' : get_log_transform,
                            'GarageQual' : get_one_hot_encoded_transformation,
                            'GarageCond' : get_one_hot_encoded_transformation,
                            'PavedDrive' : get_one_hot_encoded_transformation,
                            'WoodDeckSF' : get_log_transform, 
                            'OpenPorchSF' : get_log_transform,
                            'EnclosedPorch' : get_log_transform,
                            '3SsnPorch' : get_log_transform,
                            'ScreenPorch' : get_log_transform, 
                            'PoolArea' : get_log_transform, 
                            'PoolQC' : get_one_hot_encoded_transformation,
                            'Fence' : get_one_hot_encoded_transformation,
                            'MiscFeature' : get_one_hot_encoded_transformation,
                            'MiscVal' : get_log_transform, 
                            'MoSold' : get_one_hot_encoded_transformation, 
                            'YrSold' : get_one_hot_encoded_transformation, 
                            'SaleType' : get_one_hot_encoded_transformation,
                            'SaleCondition' : get_one_hot_encoded_transformation,
                            'Alley' : get_nothing}
    rel_X_cols = get_rel_X_cols(
        X_columns=rel_cols,
        X_column_transform_map=relevant_transform_map,
        input_df=train_data_one_hot,
        validation_df=validation_data_one_hot)
    
    other_cols = [col for col in rel_X_cols if col != given_col]
    X = train_validation_data_one_hot[other_cols]
    Y = train_validation_data_one_hot[[given_col]]
    my_model = linear_model.LinearRegression()
    my_model.fit(X, Y)
    return (my_model, my_model.score(X, Y))


In [16]:
(my_model, r_squared) = get_vif_model(given_col='MSSubClass_120')

In [17]:
r_squared

1.0

#### No effect !

We should have forseen this before. Though we do not have linearly dependent predictors, the dependent variable is a plain linear combination of some of the predictors , causing us to have an R Squared of 1.

Hence let us make sure that we remove two of the one hot encoded columns , as opposed to just one.

In [18]:
def get_one_hot_encoded_transformation(column, input_df=None, validation_df=None):
    
    rel_cols = [col for col in input_df.columns if column + '_' in col and column + '_Val' not in col]
    # Omit one column while returning the column list to avoid linear dependence amonst columns.
    return rel_cols[:-2]


In [19]:
(my_model, r_squared) = get_vif_model(given_col='MSSubClass_120')
r_squared


0.9800570650678383

This illustrates an issue. Because of the nature of one hot encoding, the multicollinearity will be very high, if we were to include other one hot encoded columns amongst the predictors. I believe that this rather inflates the multicollinearity than what it should be.


Let us do one thing. For categorical variables, let us include only one of the dependent variable and remove all it's other one hot encoded columns from the list of predictors and then see VIF's come out. This way, we can try to get a true sense of multicollinearity amongst predictors.

On second thought, let us refine it even further. For every categorical variable, let us build models in which one of the one hot encoded variable is the dependent variable (and , as mentioned before , omitting the other one hot encoded columns from the predictor list). Then , let us take maximum r-squared obtained amongst these models and use the same to calculate VIF. I will explain it with an example :

Let us assume that we have 5 predictors A,B,C,D,E and only A is a categorical variable amongst these (and the rest numeric). Also, let us assume that A can take values of 1,2 and 3.

==> List of predictors are A_1, A_2, A_3, C,D, E.

=> For computing VIF of A, we build the following 3 models :

        A_1 ~ (C, D , E)
        A_2 ~ (C, D , E)
        A_3 ~ (C, D , E)
        
and take the maximum of r squared obtained amongst these to compute the VIF for A.

In [20]:
def build_model_and_compute_r_squared(dependent_col, 
                                      predictor_cols, 
                                      train_validation_data_one_hot=train_validation_data_one_hot):

    X = train_validation_data_one_hot[predictor_cols]
    Y = train_validation_data_one_hot[[dependent_col]]
    my_model = linear_model.LinearRegression()
    my_model.fit(X, Y)
    return (my_model, my_model.score(X, Y))


In [21]:
def get_vifs(X_columns, 
             X_column_transform_map, 
             full_cols = rel_X_cols,
             train_validation_data_one_hot=train_validation_data_one_hot):
    rel_X_cols = list()
    col_to_max_r_squared = dict()
    for col in X_columns:
        if col in X_column_transform_map.keys():
            rel_cols = X_column_transform_map.get(col)(col , train_validation_data_one_hot)  
        else:
            rel_cols = [col]
        r_squareds = list()
        for elem in rel_cols:
            dep_col  = elem
            pred_cols = [x_col for x_col in full_cols if col not in x_col]

            #  Since we had merged train and test dataframes , prior to one hot encoding, we could find
            # cases of zero columns, which would yield an R Squared of 1. Hence, we ignore those.
            r_squared = build_model_and_compute_r_squared(dep_col, 
                                                          pred_cols, 
                                                          train_validation_data_one_hot)[1]
            if r_squared != 1:
                r_squareds.append(r_squared)
        if len(r_squareds) >= 1:
            col_to_max_r_squared[col] = min(r_squareds)
        for elem in rel_cols:
            rel_X_cols.append(elem)
    return col_to_max_r_squared

In [22]:
#col_to_max_r_squared = get_vifs(rel_cols, relevant_transform_map)

In [23]:
#sorted(col_to_max_r_squared.items(), key = lambda x : x[1], reverse=True)

### Issues here.

1. We have the following issues with testing of multicollinearity for categorical variables. 

    (a) Since we merged training and test data before doing one hot encoding, we occassionally find 0 columns, which shown an R squared of 1
   
   (b) Even though, we take care of the cases mentioned above, we are having extremely sparse columns, which can shown an abnormally high R Squared (and hence VIF). Though, we could take care of this, by taking the minium R Squared obtained for each categorical variable(amongst the one hot encoded columns), I am not sure if that is the ideal way to do it.



2. From the stats seen above, there does look to be a lot of multicollinearity (even if you ignore the categorical variables), but how do we move forward in this case >


### Thoughts

1. Doing an outright PCA does not make sense due to inflation in multicollinearity shown by extremely sparse one hot encoded column.

2. One option would be to do PCA only on the numerical columns. This is feasible, but given the extremely large number of categorical columns present, how do we decide to merge the result of this PCA on numeric predictors with other categorical ones ? How do we do a dimensionality reduction on the categorical predictors ?



### Action 

One action would be to do a method which works on a mixture of both quantitative and categorical variables, namely Factor Analysis on Mixed data (See https://stats.stackexchange.com/questions/5774/can-principal-component-analysis-be-applied-to-datasets-containing-a-mix-of-cont).

Fortunately , we have a python implementation and let us see if that helps (https://github.com/MaxHalford/Prince#factor-analysis-of-mixed-data-famd). 

Let us check it out !

Now, that we are using a method, which deals with categorical variables as it is, we do not need to do one hot encoding on them.

Hence, we come up with a new transformation, replacing the one hot encoding transformation with identity transformation.

In [24]:
famd_mthd_transform_map={'Id' : get_nothing,
                        'MSSubClass' : get_column_itself,
                        'MSZoning' : get_column_itself,
                        'LotFrontage': get_nothing,
                        'LotArea': get_log_transform,
                        'Street' : get_column_itself,                            
                        'LotShape' : get_column_itself,                                                     
                        'LandContour' : get_column_itself,                                                     
                        'Utilities' : get_column_itself,                                                                                 
                        'LotConfig' : get_column_itself,                                                                                                             
                        'LandSlope' : get_column_itself,                                                                                                                                         
                        'Neighborhood' : get_column_itself,                                                                                                                                                                     
                        'Condition1' : get_column_itself,                                                                                                                                                                                                 
                        'Condition2' : get_column_itself,                                                                                                                                                                                                 
                        'BldgType' : get_column_itself,                                                                                                                                                                                                 
                        'HouseStyle' : get_column_itself,  
                        'OverallQual' : get_column_itself,  
                        'OverallCond' : get_column_itself,                              
                        'YearBuilt' : get_column_itself,  
                        'YearRemodAdd' : get_column_itself,                              
                        'RoofStyle' : get_column_itself,                                                                                                                                                                     
                        'RoofMatl' : get_column_itself,                                                                                                                                                                                                 
                        'Exterior1st' : get_column_itself,                                                                                                                                                                                                 
                        'Exterior2nd' : get_column_itself,                                                                                                                                                                                                 
                        'MasVnrType' : get_column_itself,  
                        'MasVnrArea' : get_null_value_handling_with_log_transform,
                        'ExterQual' : get_column_itself,
                        'ExterCond' : get_column_itself,
                        'Foundation' : get_column_itself,
                        'BsmtQual' : get_column_itself,
                        'BsmtCond' : get_column_itself,
                        'BsmtExposure' : get_column_itself,
                        'BsmtFinType1' : get_column_itself,   
                        'BsmtFinSF1' : get_log_transform,
                        'BsmtFinType2' : get_column_itself,
                        'BsmtUnfSF' : get_column_itself, 
                        'TotalBsmtSF' : get_column_itself,
                        'Heating' : get_column_itself,
                        'HeatingQC' : get_column_itself,
                        'CentralAir' : get_column_itself,
                        'Electrical' : get_column_itself,  
                        '1stFlrSF' : get_log_transform,
                        '2ndFlrSF' : get_log_transform,
                        'LowQualFinSF' : get_log_transform,
                        'GrLivArea' : get_log_transform,
                        'BsmtFullBath' : get_column_itself, 
                        'BsmtHalfBath':get_column_itself, 
                        'FullBath' : get_column_itself,
                        'HalfBath' : get_column_itself, 
                        'BedroomAbvGr' : get_column_itself, 
                        'KitchenAbvGr' :get_column_itself,
                        'KitchenQual' : get_column_itself,
                        'TotRmsAbvGrd' : get_column_itself,
                        'Functional' : get_column_itself, 
                        'Fireplaces' : get_column_itself,  
                        'FireplaceQu' : get_column_itself,
                        'GarageType' : get_column_itself,
                        'GarageYrBlt' : get_null_value_handling_with_no_transform,
                        'GarageFinish' : get_column_itself,
                        'GarageCars': get_column_itself,
                        'GarageArea' : get_log_transform,
                        'GarageQual' : get_column_itself,
                        'GarageCond' : get_column_itself,
                        'PavedDrive' : get_column_itself,
                        'WoodDeckSF' : get_log_transform,
                        'OpenPorchSF' : get_log_transform,
                        'EnclosedPorch' : get_log_transform, 
                        '3SsnPorch' : get_log_transform,
                        'ScreenPorch' : get_log_transform, 
                        'PoolArea' : get_log_transform, 
                        'PoolQC' : get_column_itself,
                        'Fence' : get_column_itself,
                        'MiscFeature' : get_column_itself,
                        'MiscVal' : get_log_transform, 
                        'MoSold' : get_column_itself, 
                        'YrSold' : get_column_itself, 
                        'SaleType' : get_column_itself,
                        'SaleCondition' : get_column_itself,
                        'Alley' : get_nothing}


In [25]:
rel_X_cols = get_rel_X_cols(
    X_columns=rel_cols,
    X_column_transform_map=famd_mthd_transform_map,
    input_df=train_data)



In [26]:
rel_X_cols

['MSSubClass',
 'MSZoning',
 'LogLotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'LogMasVnrArea_times_not_missing',
 'MasVnrArea_not_missing',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'LogBsmtFinSF1',
 'BsmtFinType2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'Log1stFlrSF',
 'Log2ndFlrSF',
 'LogLowQualFinSF',
 'LogGrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt_times_not_missing',
 'GarageYrBlt_not_missing',
 'GarageFinish',
 'GarageCars',
 'LogGarageArea',
 'Garag

Now, let us take a stab at FAMD method and see how it works.

In [27]:
columns_for_famd = [
 'MSSubClass',
 'MSZoning',
 'LogLotArea',
 'Street',
 'LotShape',
 'LandContour', 
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'LogMasVnrArea_times_not_missing',
 'MasVnrArea_not_missing',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'LogBsmtFinSF1',
 'BsmtFinType2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'Log1stFlrSF',
 'Log2ndFlrSF',
 'LogLowQualFinSF',
 'LogGrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt_times_not_missing']

In [28]:
columns_for_famd = [
 'MSSubClass',
 'LogGrLivArea',
 'MSZoning',    
 'LogLotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt', 
 'YearRemodAdd',
 'LogLowQualFinSF'
]

In [29]:

columns_for_famd = [
 'MSSubClass',
 'LogGrLivArea',
 'MSZoning',    
]


In [30]:
test_data = pd.read_csv('../../input/test.csv')

In [31]:
train_data_for_famd = train_data[columns_for_famd].copy()
validation_data_for_famd = validation_data[columns_for_famd].copy()
train_validation_data_for_famd = train_validation_data[columns_for_famd].copy()
test_data['LogGrLivArea'] = test_data_one_hot['LogGrLivArea']
test_data['LogLotArea'] = test_data_one_hot['LogLotArea']
test_data['LogMasVnrArea_times_not_missing'] = test_data_one_hot['LogMasVnrArea_times_not_missing']
test_data['LogBsmtFinSF1'] = test_data_one_hot['LogBsmtFinSF1']
test_data['Log1stFlrSF'] = test_data_one_hot['Log1stFlrSF']
test_data['Log2ndFlrSF'] = test_data_one_hot['Log2ndFlrSF']
test_data_for_famd = test_data[columns_for_famd].copy()

In [32]:
train_data_for_famd.shape

(1095, 3)

In [33]:
validation_data_for_famd.shape

(365, 3)

In [34]:
test_data_for_famd.shape

(1459, 3)

In [35]:
train_data_for_famd['MSSubClass'] = train_data_for_famd['MSSubClass'].apply(lambda x : '#' + str(x) + '#')
validation_data_for_famd['MSSubClass'] = validation_data_for_famd['MSSubClass'].apply(lambda x : '#' + str(x) + '#')
test_data_for_famd['MSSubClass'] = test_data_for_famd['MSSubClass'].apply(lambda x : '#' + str(x) + '#')
train_validation_data_for_famd['MSSubClass'] = train_validation_data_for_famd['MSSubClass'].apply(lambda x : '#' + str(x) + '#')
#train_data_for_famd['MasVnrArea_not_missing'] = \
#    train_data_for_famd['MasVnrArea_not_missing'].apply(lambda x : '#' + str(x) + '#')


In [36]:
train_data_for_famd['MSSubClass'].describe()

count     1095
unique      15
top       #20#
freq       403
Name: MSSubClass, dtype: object

In [37]:
def build_and_test_famd_model(num_components=25,
                              train_data_for_famd=train_data_for_famd, 
                              Y = train_data['LogSalePrice'].values.ravel(),
                              validation_data_for_famd=validation_data_for_famd
                             ):
    famd = prince.FAMD(
        n_components=num_components,
        n_iter=20,
        copy=True,
        check_input=True, 
        engine='auto',
        random_state=42)
    famd_model = famd.fit(train_data_for_famd)
    transformed_X = famd_model.transform(train_data_for_famd)
    print(transformed_X.shape)
    my_model = linear_model.LinearRegression()
    my_model.fit(transformed_X, Y)
    predictions = my_model.predict(transformed_X)
    
    print(validation_data_for_famd.shape)    

    transformed_X_valid = famd_model.transform(validation_data_for_famd)    

    predictions_valid = my_model.predict(transformed_X_valid)    
    return (my_model, 
            famd_model,
            predictions,
            predictions_valid)   
      

In [38]:
import prince

In [39]:
print(train_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       ...
       'LogGarageArea', 'LogWoodDeckSF', 'LogOpenPorchSF', 'LogEnclosedPorch',
       'Log3SsnPorch', 'LogScreenPorch', 'LogPoolArea', 'LogMiscVal',
       'LogSalePrice', 'LogMasVnrArea_times_not_missing'],
      dtype='object', length=102)


In [40]:
train_validation_data[['Condition1']].groupby(['Condition1']).size()

Condition1
Artery      48
Feedr       81
Norm      1260
PosA         8
PosN        19
RRAe        11
RRAn        26
RRNe         2
RRNn         5
dtype: int64

In [41]:
train_validation_data[['RoofStyle']].groupby(['RoofStyle']).size()

RoofStyle
Flat         13
Gable      1141
Gambrel      11
Hip         286
Mansard       7
Shed          2
dtype: int64

In [42]:
train_validation_data['Condition2'].unique()

array(['Norm', 'Artery', 'RRNn', 'Feedr', 'PosN', 'PosA', 'RRAn', 'RRAe'],
      dtype=object)

In [43]:
(my_model, famd_model, predicitons_train_data, predictions_validation_data) = \
    build_and_test_famd_model(num_components=25)

train_score = evaluate_model_score_given_predictions(predicitons_train_data, 
                                                     train_data['LogSalePrice'].values.ravel())
validation_score = evaluate_model_score_given_predictions(predictions_validation_data,
                                                          validation_data['LogSalePrice'].values.ravel())
print(train_score, validation_score)

(1095, 21)
(365, 3)


ValueError: shapes (365,20) and (21,21) not aligned: 20 (dim 1) != 21 (dim 0)

In [ ]:
famd = prince.FAMD(
    n_components=25,
    n_iter=20,
    copy=True,
    check_input=True, 
    engine='auto',
    random_state=42)



In [ ]:
famd_model = famd.fit(train_data_for_famd)
transformed_X = famd_model.transform(train_data_for_famd)


In [ ]:
Y = train_data['LogSalePrice'].values.ravel()

In [ ]:
print(transformed_X.shape)
my_model = linear_model.LinearRegression()
my_model.fit(transformed_X, Y)
predictions = my_model.predict(transformed_X)

print(validation_data_for_famd.shape)    

transformed_X_valid = famd_model.transform(validation_data_for_famd)    

predictions_valid = my_model.predict(transformed_X_valid)    


In [ ]:
transformed_X.mean()

In [ ]:
transformed_X_valid.mean()

In [ ]:
validation_data['LogSalePrice'].values.ravel()

In [ ]:
len(famd_model.explained_inertia_)

In [ ]:
sum(famd_model.explained_inertia_)

### Make sure that we are able to make predictions on test data as well

In [ ]:
test_data['MSZoning'].describe()

In [ ]:
columns_for_famd

In [ ]:
test_data['MSSubClass'].unique()


In [ ]:
train_validation_data['MSSubClass'].unique()

In [ ]:
test_data[test_data['MSSubClass'] == 150]

In [ ]:
print(len(test_data_for_famd[test_data_for_famd['MSSubClass'] != 150]))

In [ ]:
print(len(test_data_for_famd))

In [ ]:
print(len(test_data_for_famd[test_data_for_famd['MSSubClass'] != '#150#']))

In [ ]:
#There is one value for MSSubClass that is present in test set and not present in training set. 
#Let us delete this entry for now.C
test_data_for_famd_1 = test_data_for_famd[test_data_for_famd['MSSubClass'] != '#150#']
test_data_for_famd_2 =  test_data_for_famd_1[test_data_for_famd_1['MSZoning'].notnull()]
(my_model, famd_model, predicitons_train_data, predictions_test_data) = \
    build_and_test_famd_model(num_components=25,
                              train_data_for_famd=train_validation_data_for_famd, 
                              Y = train_validation_data['LogSalePrice'].values.ravel(),
                              validation_data_for_famd=test_data_for_famd_2)
train_score = evaluate_model_score_given_predictions(predicitons_train_data, 
                                                     train_validation_data['LogSalePrice'].values.ravel())
print(train_score)

There is one value for MSSubClass that is present in test set and not present in training set. Let us delete this entry for now.

### Build a cross validation routine.

NOTE: I have written a routine here and though it theoretically works well, practically it causes lots of problems with some categorical variables having few values in the training set and not in the test set (and vice versa). This situation causes the FAMD routine to given unreliable results

Also, there is another issue https://github.com/MaxHalford/prince/issues/61, which causes some of  the iterations to show absurd results. Hence, we need to be aware of the same.


In [ ]:
def get_trained_model_and_transform(X, Y, num_components=25, num_iter=20):
    famd = prince.FAMD(
        n_components=num_components,
        n_iter=num_iter,
        copy=True,
        check_input=True, 
        engine='auto',
        random_state=42)

    famd_model = famd.fit(X)
    transformed_X = famd_model.transform(X)
    
    my_model = linear_model.LinearRegression()
    
    my_model.fit(transformed_X,Y)

    return (my_model, famd_model)

In [ ]:
def get_cross_val_output(input_df, 
                         num_components=25, 
                         num_iter=20,
                         nfolds=5):
    partition_indices = np.array_split(np.arange(len(input_df)), nfolds)
    Y_column = 'LogSalePrice'
    cross_validated_scores = np.zeros(nfolds)
    rel_X_cols = [col for col in input_df.columns if col != Y_column]
    cross_validated_data = pd.DataFrame(columns=input_df.columns)
    for i in range(nfolds):
        cross_validated_set = input_df[partition_indices[i][0]:partition_indices[i][-1] + 1].copy()
        rel_training_data = pd.DataFrame(columns=input_df.columns, dtype=float)
        for j in range(nfolds):
            if j != i:
                training_set = input_df[partition_indices[j][0]:partition_indices[j][-1] + 1]
                rel_training_data = pd.concat([rel_training_data, training_set])
        (my_model, famd_model) = get_trained_model_and_transform(rel_training_data[rel_X_cols], 
                                                                 rel_training_data[[Y_column]].values.ravel(),
                                                                 num_components=num_components,
                                                                 num_iter=num_iter)
        '''
        famd = prince.FAMD(
            n_components=num_components,
            n_iter=num_iter,
            copy=True,
            check_input=True, 
            engine='auto',
            random_state=42)
        X = rel_training_data[rel_X_cols]
        Y = rel_training_data[[Y_column]].values.ravel()
        
        famd_model = famd.fit(X)
        transformed_X = famd_model.transform(X)

        my_model = linear_model.LinearRegression()

        my_model.fit(transformed_X,Y)
        '''
        newX = cross_validated_set[rel_X_cols]
        newY = cross_validated_set[[Y_column]]
        
        new_transformed_X = famd_model.transform(newX)
        new_predictions = my_model.predict(new_transformed_X)
        
        cross_validated_score = evaluate_model_score_given_predictions(new_predictions, newY)
        cross_validated_scores[i] = cross_validated_score
        if i==3:
            #print(cross_validated_score)
            #rel_training_data.to_csv('data\cv_training3.csv', index=False)
            #cross_validated_set.to_csv('data\cv_validation3.csv', index=False)            
            (my_model, famd_model) = get_trained_model_and_transform(rel_training_data[rel_X_cols], 
                                                                     rel_training_data[[Y_column]].values.ravel(),
                                                                     num_components=num_components,
                                                                     num_iter=num_iter)
            newX = cross_validated_set[rel_X_cols]
            newY = cross_validated_set[[Y_column]]

            new_transformed_X = famd_model.transform(newX)
            new_predictions = my_model.predict(new_transformed_X)
            print(new_predictions.mean())
            cross_validated_score = evaluate_model_score_given_predictions(new_predictions, newY)
            print(cross_validated_score)
            
        
    return cross_validated_scores

In [ ]:
def evaluate_model_score_with_transform(my_model, famd_model, newX, newY):
    new_transformed_X = famd_model.transform(newX)
    
    return evaluate_model_score(my_model, new_transformed_X, newY)

In [ ]:
relevnat_cols_for_cross_val = columns_for_famd.copy()
relevnat_cols_for_cross_val.append('LogSalePrice')


In [ ]:
relevnat_cols_for_cross_val

In [ ]:
train_validation_data_for_famd_cv = train_validation_data_for_famd.copy()
train_validation_data_for_famd_cv['LogSalePrice'] = train_validation_data['LogSalePrice']

In [ ]:
#temp_df = train_validation_data_for_famd_cv.sample(frac=1)
scores = get_cross_val_output(train_validation_data_for_famd_cv.sample(frac=1), 
                              num_components=25)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
scores = get_cross_val_output(temp_df, 
                              num_components=25)

In [ ]:
scores

In [ ]:
cv_training = pd.read_csv('data/cv_training.csv')


In [ ]:
scores = get_cross_val_output(cv_training, 
                              num_components=25)

In [ ]:
scores